In [1]:
%reload_ext autoreload
%autoreload 2

import utils
import models
import datasets

#!/usr/bin/python3

import argparse
import itertools

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from PIL import Image
import torch

from models import Generator
from models import Discriminator
from utils import ReplayBuffer
from utils import LambdaLR
from utils import Logger
from utils import weights_init_normal
from datasets import ImageDataset

import hwr
from matplotlib import pyplot as plt
import numpy as np
import warnings

In [2]:
import shlex

parser = argparse.ArgumentParser()
parser.add_argument('--epoch', type=int, default=0, help='starting epoch')
parser.add_argument('--n_epochs', type=int, default=200, help='number of epochs of training')
parser.add_argument('--batchSize', type=int, default=1, help='size of the batches')
#parser.add_argument('--dataroot', type=str, default='datasets/horse2zebra/', help='root directory of the dataset')
parser.add_argument('--lr', type=float, default=0.0002, help='initial learning rate')
parser.add_argument('--decay_epoch', type=int, default=100, help='epoch to start linearly decaying the learning rate to 0')
parser.add_argument('--sizeX', type=int, default=256, help='size of the data crop (squared assumed)')
parser.add_argument('--sizeY', type=int, default=256, help='size of the data crop (squared assumed)')
parser.add_argument('--input_nc', type=int, default=3, help='number of channels of input data')
parser.add_argument('--output_nc', type=int, default=3, help='number of channels of output data')
parser.add_argument('--cuda', action='store_true', help='use GPU computation')
parser.add_argument('--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation')
parser.add_argument('--online_root', type=str, default='datasets/horse2zebra/trainA', help='root directory of the dataset')
parser.add_argument('--offline_root', type=str, default='datasets/horse2zebra/trainB', help='root directory of the dataset')


#opt = parser.parse_args()


_StoreAction(option_strings=['--offline_root'], dest='offline_root', nargs=None, const=None, default='datasets/horse2zebra/trainB', type=<class 'str'>, choices=None, help='root directory of the dataset', metavar=None)

In [3]:
if True:
    cycle_loss = True
    identity_loss = True
    B2A = True
    batch = 2
else:
    cycle_loss = False
    identity_loss = False
    B2A = False
    batch = 12
    
args = f"--epoch 0 --cuda --input_nc 1 --output_nc 1 --batchSize {batch} --sizeX 64 --sizeY 1280 --online_root /media/data/GitHub/handwriting_data/train_online_cropped --offline_root /media/data/GitHub/handwriting_data/train_offline_preprocessed "
opt = parser.parse_args(shlex.split(args))
torch.cuda.empty_cache()

if torch.cuda.is_available() and not opt.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

device = "cuda" if opt.cuda else "cpu"
###### Definition of variables ######
# Networks
print("Creating newtorks")

netG_A2B = Generator(opt.input_nc, opt.output_nc)
netG_B2A = Generator(opt.output_nc, opt.input_nc)
netD_A = Discriminator(opt.input_nc)
netD_B = Discriminator(opt.output_nc)

if opt.cuda:
    netG_A2B.cuda()
    netG_B2A.cuda()
    netD_A.cuda()
    netD_B.cuda()

netG_A2B.apply(weights_init_normal)
netG_B2A.apply(weights_init_normal)
netD_A.apply(weights_init_normal)
netD_B.apply(weights_init_normal)

# Losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

print("Creating optimizers...")
# Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(itertools.chain(netG_A2B.parameters(), netG_B2A.parameters()),
                                lr=opt.lr, betas=(0.5, 0.999))
optimizer_D_A = torch.optim.Adam(netD_A.parameters(), lr=opt.lr, betas=(0.5, 0.999))
optimizer_D_B = torch.optim.Adam(netD_B.parameters(), lr=opt.lr, betas=(0.5, 0.999))

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)

print("Creating inputs...")
# Inputs & targets memory allocation
Tensor = torch.cuda.FloatTensor if opt.cuda else torch.Tensor
input_A = Tensor(opt.batchSize, opt.input_nc, opt.sizeX, opt.sizeY)
input_B = Tensor(opt.batchSize, opt.output_nc, opt.sizeX, opt.sizeY)
target_real = Variable(Tensor(opt.batchSize).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(opt.batchSize).fill_(0.0), requires_grad=False)

fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

print("Creating dataloaders....")
# Dataset loader
#                 transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
transforms_ = [ transforms.ToTensor()]
dataloader = DataLoader(ImageDataset(opt.online_root, opt.offline_root, transforms_=transforms_, unaligned=True), 
                        batch_size=opt.batchSize, shuffle=True, num_workers=opt.n_cpu)

# Loss plot
logger = Logger(opt.n_epochs, len(dataloader))
###################################



Creating newtorks


/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/site-packages/torch/cuda/__init__.py:118: UserWarning: 
    Found GPU1 GeForce GTX 660 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))
/media/data/GitHub/PyTorch-CycleGAN/utils.py:121: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight.data, 0.0, 0.02)


Creating optimizers...
Creating inputs...
Creating dataloaders....


In [4]:
def plot_tensors(tensor):
    for i in range(0, tensor.shape[0]):
        plot_tensor(tensor[i,0])

def plot_tensor(tensor):
    print(tensor.shape)
    t = tensor.cpu()
    assert not np.isnan(t).any()
    plt.figure(dpi=400)
    plt.imshow(t, cmap='gray')
    plt.show()

In [5]:
use_hw_loss = False
hw = hwr.HWR(batch_size=opt.batchSize)
zero_tensor = torch.tensor([0.0], requires_grad=False).to(device)

###### Training ######
print("Starting first epoch")
for epoch in range(opt.epoch, opt.n_epochs):
    print(f"Epoch {epoch}")
    for i, batch in enumerate(dataloader):
        # Set model input
        real_A = Variable(input_A.copy_(batch['A']))
        real_B = Variable(input_B.copy_(batch['B']))
               
        # Label
        if use_hw_loss:
            t = hw.hwr_predict(real_A[:,:,2:62,:], as_string=False)
            label_A, label_A_lengths = hwr.build_labels(t, device)

            # Ensure HWR is working:
            if False:
                plot_tensors(real_A)
                string_prediction = hw.hwr_predict(real_A[:,:,2:62,:])
                print(string_prediction)
                input()
        
        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        if identity_loss:
            # G_A2B(B) should equal B if real B is fed
            same_B = netG_A2B(real_B)
            loss_identity_B = criterion_identity(same_B, real_B)*5.0
            # G_B2A(A) should equal A if real A is fed
            same_A = netG_B2A(real_A)
            loss_identity_A = criterion_identity(same_A, real_A)*5.0
        else:
            loss_identity_A = zero_tensor
            loss_identity_B = zero_tensor
            
        # GAN loss
        fake_B = netG_A2B(real_A)
        pred_fake = netD_B(fake_B)
        loss_GAN_A2B = criterion_GAN(pred_fake, target_real)

        if B2A:
            fake_A = netG_B2A(real_B)
            pred_fake = netD_A(fake_A)
            loss_GAN_B2A = criterion_GAN(pred_fake, target_real)
        else:
            loss_GAN_B2A = zero_tensor
            fake_A = zero_tensor
            
            
        # Cycle loss
        if cycle_loss:
            recovered_A = netG_B2A(fake_B)
            loss_cycle_ABA = criterion_cycle(recovered_A, real_A)*10.0

            recovered_B = netG_A2B(fake_A)
            loss_cycle_BAB = criterion_cycle(recovered_B, real_B)*10.0
        else:
            loss_cycle_BAB = zero_tensor
            loss_cycle_ABA = zero_tensor
        
#         # Handwriting loss
#         hw_loss = hw.hwr_loss(real_A[:,:,2:62,:], label_A, label_A_lengths)
#         string_prediction = hw.hwr_predict(real_A[:,:,2:62,:], as_string=False)
#         print(f"Real Prediction {string_prediction}")
#         print(f"HW loss REAL: {hw_loss}")

#         string_prediction = hw.hwr_predict(fake_B[:,:,2:62,:], as_string=False)
#         print(f"Fake Prediction {string_prediction}")
#         hw_loss = hw.hwr_loss(fake_B[:,:,2:62,:], label_A, label_A_lengths)
#         print(f"HW loss FAKE: {hw_loss}")
#         print(label_A, label_A_lengths)
#         Stop
        
        if use_hw_loss:
            hw_loss = hw.hwr_loss(fake_B[:,:,2:62,:], label_A, label_A_lengths) / 10
            if torch.isinf(hw_loss):
                warnings.warn(f"Encountered {hw_loss} hw_loss")
                hw_loss = zero_tensor
        else:
            hw_loss = zero_tensor
        # np.nanmax(np.isfinite([hw_loss,0]))
        
        # Total loss
        loss_G = loss_identity_A + loss_identity_B + loss_GAN_A2B + loss_GAN_B2A + loss_cycle_ABA + loss_cycle_BAB + hw_loss
        loss_G.backward()
        
        optimizer_G.step()
        
        if np.isnan(loss_G.item()):
            for i in range(0,real_A.shape[0]):
                plot_tensor(real_A[i,0])
                plot_tensor(real_B[i,0])
                
            Stop
        
        ###################################

        ###### Discriminator A ######
        optimizer_D_A.zero_grad()

        # Real loss
        pred_real = netD_A(real_A)
        loss_D_real = criterion_GAN(pred_real, target_real)

        # Fake loss
        if B2A:
            fake_A = fake_A_buffer.push_and_pop(fake_A)
            pred_fake = netD_A(fake_A.detach())
            loss_D_fake = criterion_GAN(pred_fake, target_fake)
        else:
            loss_D_fake = zero_tensor
            
        # Total loss
        loss_D_A = (loss_D_real + loss_D_fake)*0.5
        loss_D_A.backward()

        optimizer_D_A.step()
        ###################################

        ###### Discriminator B ######
        optimizer_D_B.zero_grad()

        # Real loss
        pred_real = netD_B(real_B)
        loss_D_real = criterion_GAN(pred_real, target_real)
        
        # Fake loss
        fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_B(fake_B.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)

        # Total loss
        loss_D_B = (loss_D_real + loss_D_fake)*0.5
        loss_D_B.backward()

        optimizer_D_B.step()
        ###################################

        # Progress report (http://localhost:8097)
        logger.log({'loss_G': loss_G, 'loss_G_identity': (loss_identity_A + loss_identity_B), 'loss_G_GAN': (loss_GAN_A2B + loss_GAN_B2A),
                    'loss_G_cycle': (loss_cycle_ABA + loss_cycle_BAB), 'loss_D': (loss_D_A + loss_D_B), 'hw_loss':hw_loss}, 
                    images={'real_A': real_A, 'real_B': real_B, 'fake_A': fake_A, 'fake_B': fake_B})

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    # Save models checkpoints
    torch.save(netG_A2B.state_dict(), 'output/netG_A2B.pth')
    torch.save(netG_B2A.state_dict(), 'output/netG_B2A.pth')
    torch.save(netD_A.state_dict(), 'output/netD_A.pth')
    torch.save(netD_B.state_dict(), 'output/netD_B.pth')
###################################


/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'crnn.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.activation.ReLU' has changed. you can retrieve the original sour

Starting first epoch
Epoch 0


/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 001/200 [0286/6598] -- loss_G: 2.4147 | loss_G_identity: 0.5284 | loss_G_GAN: 0.7876 | loss_G_cycle: 1.0987 | loss_D: 0.5021 | hw_loss: 0.0000 -- ETA: 10 days, 9:28:45.433765

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/taylor/anaconda3/envs/cyclegan/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + 

KeyboardInterrupt: 

In [ ]:
real_A.shape

In [ ]:
import torch
x = torch.tensor([[0],[0]]).repeat(1,4)
print(x)

In [ ]:
preds_size = Variable(torch.IntTensor([x.size(0)] * x.size(1)))
print(preds_size)

In [ ]:
ctc = torch.nn.CTCLoss()
log_softmax = torch.nn.LogSoftmax(dim=2).to(device)
ctc_criterion = lambda x, y, z, t: ctc(log_softmax(x), y, z, t)


In [ ]:
print(t)
tt = torch.tensor(t).to(device)
inputs_size = Variable(torch.IntTensor([t.size(0)] * t.size(1)))
ctc_criterion(t, label_A, inputs_size, label_A_lengths)

In [ ]:
zero_tensor = torch.tensor([[0.0]], requires_grad=False).to(device)
print(zero_tensor.size())

In [ ]:
print("SUCCESS")